In [68]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: ANYWIDGET_HMR=1


In [82]:
import numpy as np
from numpy.typing import ArrayLike
import json
from traitlets import observe
from pathlib import Path
data = json.loads(Path("/Users/mpiekenbrock/Downloads/miserables.json").read_text())

node_ids = np.unique([n["id"] for n in data["nodes"]])
src_ids = np.array([e["source"] for e in data["links"]])
tgt_ids = np.array([e["target"] for e in data["links"]])

src_ids = np.searchsorted(node_ids, src_ids)
tgt_ids = np.searchsorted(node_ids, tgt_ids)
node_ids = np.searchsorted(node_ids, node_ids)

edgelist = np.c_[src_ids, tgt_ids]

import traitlets
import anywidget
class Widget(anywidget.AnyWidget):
    _esm = "pixiplex/static/widget.js"
    _css = "pixiplex/static/widget.css"
    
    width = traitlets.Int(250).tag(sync=True)
    height = traitlets.Int(250).tag(sync=True)
    
    node_ids = traitlets.List(trait=traitlets.CInt).tag(sync=True)
    src_ids = traitlets.List(trait=traitlets.CInt).tag(sync=True)
    tgt_ids = traitlets.List(trait=traitlets.CInt).tag(sync=True)

    _x = traitlets.List(trait=traitlets.Float).tag(sync=True)
    _y = traitlets.List(trait=traitlets.Float).tag(sync=True)

    node_color = traitlets.List(trait=traitlets.Unicode).tag(sync=True)
    
    def __init__(self, node_ids: ArrayLike, edgelist: ArrayLike, width: int = 250, height: int = 250) -> None:
        super().__init__()
        self.width = width 
        self.height = height
        self.node_ids = list(node_ids)
        self.src_ids = list(edgelist[:,0])
        self.tgt_ids = list(edgelist[:,1])

    @property
    def x(self):
        self.send({ "type" : "msg:sync_node_coordinates"})
        return self._x

    @x.setter
    def x(self, value: ArrayLike):
        value = np.atleast_1d(value).astype(np.float32).ravel();
        assert len(value) == len(self.node_ids)
        self._x = value

    def center(self, fit_zoom: bool = False, x: float = 0.0, y: float = 0.0):
        msg = { "type" : "msg:center", "fit": False }
        msg['x'] = x
        msg['y'] = y
        self.send(msg)

widget = Widget(node_ids, edgelist, width=750, height=450)
widget

Widget(height=450, node_ids=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,…

In [81]:
widget.center(False, (750*3) / 2, (450*3) / 2)

In [53]:
traitlets.Unicode

AttributeError: module 'traitlets' has no attribute 'String'

In [59]:
widget.send({ "type" : "msg:sync_node_coordinates"})
# widget._x

In [35]:

print(widget._x)
widget.send({ "type" : "msg:sync_node_coordinates"})
print(widget._x)

[]
[]


In [17]:
import asyncio
await asyncio.gather(widget.x())

/var/folders/0l/b3dbb2_d2bb4y3wbbfk0wt_80000gn/T/ipykernel_16472/4165577115.py:2: RuntimeWarning: coroutine 'Widget.x' was never awaited
  await asyncio.gather(widget.x())


TypeError: 'coroutine' object is not callable

In [6]:
wut = widget.send({ "type" : "msg:sync_node_coordinates"})
print(wut)

None


In [17]:
widget.send({ "type" : "msg:init_graph"})

In [60]:
widget.send({ "type" : "msg:init_force"})

In [61]:
widget.send({ "type" : "msg:center_graph"})

In [5]:
widget.send({ "type" : "msg:custom", "foo":"bar" })

In [ ]:
# %% Embed statically -- this works!
# import ipywidgets.embed
# ipywidgets.embed.embed_minimal_html("forcegraph_embed.html", views=[widget], drop_defaults=False)